# Schnorr digital signatures and verification
## Key pair generation

- $E$ is an elliptic curve, $N$ is the order of the curve
- $G$ is a generator and $h$ is an hash function
- The user chooses his secret key $d$, $0 < d < N$
- The user computes the point $P = dG = (P_x, P_y)$: his public key is the $x$-coordinate, $P_x$

In [1]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))

ckp.create_keypair(n_keys)["users"]

Insert the number of keys to be generated:2


[{'privateKey': '1e21897e1468d6a68d4c75c8ff5e673a4450301ffb09e9a7580fa5981d525ec7',
  'publicKey': '043b413abc2cd7e62dd213e7b0ccc70a43ab7fac6345485de5e2e5816c499b0f'},
 {'privateKey': '5c73d335ef5061a2cc38b0a26ecdac5e8f0f6c85d5f5e979c2278b0a5513d0a2',
  'publicKey': '18400974a3628b2f83dde963c7d7121e32359618df826ce1e75df8453497df8f'}]

## Schnorr Sign

- $M$ message, $d$ is the private key, the $x$-coordinate of $P = dG$ is the public key
- $A$, the signer, picks a random $k$, $1 < k < n$ and calculates $R = kG$
- A computes $e = h(R || P || M)$
- A computes $s = (k + ed) \mod n$
- The signature is the pair $(R_x, s)$ 

In [2]:
import create_keypair as ckp
import schnorr_lib as sl

user = ckp.create_keypair(1)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig = sl.schnorr_sign(M, user[0]["privateKey"])

print("PublicKey =",user[0]["publicKey"])
print("Signature =",sig.hex())

Insert the message to sign:hello
PublicKey = 4ffe079b154fbd6cf19226810d3a435927e157d3f88107cccd69624e6f010167
Signature = ad7368973fa3094b630844348045ed761153977f4498081bfd2a0746bd92e181ba38cee569ffecada71f3ba5c440ff8fc1a9f6ba44697f01bb679132ad00cfae


## Schnorr MuSig-1

- $L = h(P_1 || ... ||P_n)$, where n is the number of users and $P_i$ is the public key of the $i^{th}$ user
- Every user $i$ computes the quantity $a_i = h(L||P_i)$
- The aggregate $\tilde{X} = \sum\limits_{i=1}^{n} a_i P_i$ is a public parameter
- Every user chooses $r_i$ and computes $R_i = r_i G$
- Every user computes the point $R = R_1+...+R_n =(R_x,R_y)$
- Every user computes $c = h(R || \tilde{X}|| M)$
- Every user computes $s_i = r_i + cd_i a_i \mod N$, then the aggregate is $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$

In [3]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig, X = sl.schnorr_musig_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Insert the number of keys to be generated:3
Insert the message to sign:hello
Aggregated key = 65f86578dbc411486e0e48ad925416845f02211afce42e945918d9bb690a3186
Signature = 051b860dd425814043a71166ffeaea4cf731ca35fee35e9e3a9c8bf5139d3313e2a822a6680258e604a4632475cd48746c637defec2ae69879128df3e4618cbf


## Schnorr MuSig-2

- Compute $L$, $a_i = h(L||P_i )$ and $\tilde{X}$ as before
- Each user $i$ chooses $\nu$ different nonces $r_{i,1}, . . . , r_{i,ν}$ and computes the points
$R_{i,j} = r_{i,j}G$, $\forall j \in \{1,...,\nu\}$
- Compute $R_j = \sum\limits_{i=1}^{n} R_{i,j}, \; \forall j \in \{1,...,\nu\}$
- Compute the hash $b = h(R_1||...||R_{\nu}|| \tilde{X} || M)$, then compute $R = \sum\limits_{j=1}^{\nu} b^{j-1} R_{j}$
- Compute the hash $c = h(R || \tilde{X}||M)$, then every user $i$ computes
$s_i=c a_id_i+\sum\limits_{j=1}^{\nu} r_{i,j}\; b^{j−1} \mod N$
- Compute $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$.

In [4]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig, X = sl.schnorr_musig2_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Insert the number of keys to be generated:5
Insert the message to sign:hello
Aggregated key = 03b0a2f5b4a22ad187e68c30cac0ccf46bf6b2479ce2f0e35878a0e6cd081093
Signature = bca2573b0fc5fb7aef9a5af18dfc30eb28d1c399e4fce78302e15ca3d4be1fc361713184000f9a9ad5af6630457c5805886f1dbf14af600a38b17c22416bb1cd


## Verify Schnorr

- $B$, the receiver, computes the point $sG$
- $B$ computes $e = h(R || P || M)$ $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$
- if $sG = R + eP$, the signature is valid  $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$

In [5]:
import schnorr_lib as sl

M = input("Insert the message to verify:")
M = sl.sha256(M.encode())

pubkey = input("Insert the public key (or the aggregated key if MuSig was used):")
pubkey_bytes = bytes.fromhex(pubkey)

sig = input("Insert the generated sign:")
sig_bytes = bytes.fromhex(sig)

result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)

if result:
    print("The signature is VALID for this message and this public key")
else:
    print("The signature is NOT VALID for this message and this public key")

Insert the message to verify:hello
Insert the public key (or the aggregated key if MuSig was used):03b0a2f5b4a22ad187e68c30cac0ccf46bf6b2479ce2f0e35878a0e6cd081093
Insert the generated sign:bca2573b0fc5fb7aef9a5af18dfc30eb28d1c399e4fce78302e15ca3d4be1fc361713184000f9a9ad5af6630457c5805886f1dbf14af600a38b17c22416bb1cd
The signature is VALID for this message and this public key
